# BB84

Import needed modules

In [1]:
"""Module providing BB84 code"""

from sympy.physics.quantum.qubit import Qubit
import numpy as np

from util.measure_all import measure_all_oneshot
from util.util import get_random_bases, change_basis, zero, one, plus, minus, \
    get_random_bits

## Alice Part

Alice generates N random bits

In [2]:
alice_bits = [0, 1, 0, 1, 1, 0, 1, 1, 1]
print(f"Alice bits: {alice_bits}")

Alice bits: [0, 1, 0, 1, 1, 0, 1, 1, 1]


Alice picks N random basis between `Z` and `X`

In [3]:
alice_bases = ['Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z']
print(f"Alice bases: {alice_bases}")

Alice bases: ['Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z']


Then Alice encodes her random bits in the random bases

In [4]:
def bb84_alice_encode(alice_bits, alice_bases):
    """
    :param func alice_bits: fAlice bits
    :param func alice_bases: Alice bases
    :return alice_encoded: alice measured qubits
    """

    alice_encoded = []
    for bit, base in zip(alice_bits, alice_bases):
        qubit = Qubit(bit)
        if base == 'X':
            qubit = change_basis(qubit, (plus, minus))
        alice_encoded.append(qubit)
    return alice_encoded


alice_encoded = bb84_alice_encode(alice_bits, alice_bases)
print(f"Alice encoded: {alice_encoded}")

Alice encoded: [|0>, |1>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |1>]


## Eve's Part

Eve might be eavesdropping, she needs to pick N random basis
and measure Alice encoded Qubits
In doing so this is what can happen
```
                            Eve  ----------------- Bob     0.5 undetected
                     0.5 /  Z, |0>                 Z,|0>
     Alice   ---> Eve -
     Z, |0>             \             ------------ Bob
                     0.5 \          / 0.5          Z,|0>   0.25 undetcted
                          \        /
                           Eve---------
                           X, |+>,|->  \
                                        --------- Bob
                                                  Z,|1>   0.25 Detected
```                                                  
On n bit the probability the Eve is not detected is
`1 - (3/4)**n`


Measure Alice encoded qubits

In [5]:
def bb84_eve_eavesdropping(alice_encoded, eve_bases):
    """
    :param qubit list alice_sent: alice measured qubits
    :param int gen_bit_size: how many bit we want to generate
    return eavesdropped: alice eavesdropped qubits
    """

    eavesdropped = []
    for q, bb in zip(alice_encoded, eve_bases):
        qq = q
        # change base if needed
        if bb == 'X':
            qq = change_basis(q, (plus, minus))

        m = measure_all_oneshot(qq)
        eavesdropped.append(m)

    # measuring changes what alice_sent
    return eavesdropped

Alice and Bob can generate some more bits and reveal a part of those
checking that where their basis match the result does too
If this is not true Eve eavesdropped
On n bit the probability the Eve is not detected is 1 - (3/4)**n

In [6]:
def bb84_detect_eavesdropping(private_key, alice_bits, check_size):
    '''
    :param list private_key: complete private key
    :param int check_size: how many bit of the key
      should be used to check
           eavesdropping
    :param alice_bits: alice randomly
    picked bits (we use just check_size portion)
    '''
    eavesdropped = 0
    for pk, al in zip(private_key, alice_bits):
        # the bases have to match
        if pk != '-':
            check_size -= 1
            # the values too
            if pk != Qubit(al):
                eavesdropped += 1
        if check_size == 0:
            break
    return eavesdropped

## Bob's Part

Bob generates N random bases

In [7]:
bob_bases = ['Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z']
print(f"Bob bases: {bob_bases}")

Bob bases: ['Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z']


Bob measures Alice encoded qubits in his basis

In [8]:
def bb84_bob_measure(bob_bases, alice_encoded):
    """
    :param func bob_bases:  Bob bases
    :param qubit list alice_encoded: alice encoded qubits
    (might have been eavesdropped)
    return bob_measure: Bob measured qubits
    """
    bob_measure = []

    for q, bb in zip(alice_encoded, bob_bases):
        qq = q
        # change base if needed
        if bb == 'X':
            qq = change_basis(q, (plus, minus))

        m = measure_all_oneshot(qq)
        bob_measure.append(m)
    return bob_measure


bob_measure = bb84_bob_measure(bob_bases, alice_encoded)
print(f"Bob Measures: {bob_measure}")

Bob Measures: [|0>, |1>, |0>, |1>, |0>, |0>, |0>, |0>, |1>]


## Final part

Bob and Alice share the bases and keep only bits where basis match

In [9]:
def generate_private_key(alice_bases, bob_bases, bob_measure):
    pk = bob_measure
    # match bases
    for ab, bb, i in zip(alice_bases, bob_bases, range(len(alice_bases))):
        # if Alice and Bob bases match
        # than the measure is good
        if bb != ab:
            pk[i] = '-'
    return pk


private_key = generate_private_key(alice_bases, bob_bases, bob_measure)
pk_size = sum(1 for x in private_key if x != '-')
print(f"Private key: {private_key} size={pk_size}")

Private key: [|0>, '-', |0>, |1>, '-', |0>, '-', '-', |1>] size=5


In [10]:
truth = [zero, '-', zero, one, '-', zero, '-', '-', one]
assert private_key == truth

The full algorithm

In [11]:
def bb84(alice_bits, alice_bases, bob_bases, eavesdropping=False,
         check_size=0):
    """
    Run BB84 algorithm and return a private key

    :param func alice_bits: Alice bits
    :param func alice_bases: Alice bases
    :param func bob_bases: Bob bases
    :param Bool eavesdropping: simulate Eve eavesdropping
    :param int check_size: how many bit of the key should be used to check
           eavesdropping
    """
    alice_encoded = bb84_alice_encode(alice_bits, alice_bases)
    print(f"Alice Encoded: {alice_encoded}")
    eavesdropped = ""

    if eavesdropping:
        eve_bases = get_random_bases(len(alice_encoded))
        alice_encoded = bb84_eve_eavesdropping(alice_encoded, eve_bases)

    bob_measure = bb84_bob_measure(bob_bases, alice_encoded)
    print(f"Bob Measures: {bob_measure}")

    if eavesdropping:
        eavesdropped = bb84_detect_eavesdropping(private_key, alice_bits,
                                                 check_size)
        print(f"Eavesdropped: {eavesdropping}")

    pk = generate_private_key(alice_bases, bob_bases, bob_measure)
    print(f"pk: {pk}")

    return pk, eavesdropped

## Eavesdropping example

In [12]:
def bb84_eavesdropping_probability_detection():
    key_size = 256
    check_size = 50
    N = key_size + check_size

    alice_bits = get_random_bits(N)
    alice_bases = get_random_bases(N)
    bob_bases = get_random_bases(N)

    private_key, eavesdropped = bb84(alice_bits, alice_bases, bob_bases,
                                     eavesdropping=True, check_size=check_size)
    prob = 1. - (3 / 4) ** check_size
    print(f"prob to catch Eve: {prob} with {check_size} bits")
    print(f"Eve was caught {eavesdropped} times")

    usable_pk = sum(1 if x != '-' else 0 for x in private_key)
    print(f"Usable key len {usable_pk}")


bb84_eavesdropping_probability_detection()


Alice Encoded: [sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |1>, |1>, |1>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |1>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, |1>, |0>, |1>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, |1>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |0>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, |0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*

## Tests

In [13]:
def test_0():
    alice_bits = [0, 1, 0, 1, 1, 0, 1, 1, 1]

    alice_bases = ['Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z']

    bob_bases = ['Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z']

    pk, _ = bb84(alice_bits, alice_bases, bob_bases)
    truth = [zero, '-', zero, one, '-', zero, '-', '-', one]

    assert pk == truth


test_0()

Alice Encoded: [|0>, |1>, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |1>]
Bob Measures: [|0>, |1>, |0>, |1>, |1>, |0>, |0>, |1>, |1>]
pk: [|0>, '-', |0>, |1>, '-', |0>, '-', '-', |1>]


In [14]:
def test_1():
    np.random.seed(10)

    N = 10
    alice_bits = get_random_bits(N)
    alice_bases = get_random_bases(N)
    bob_bases = get_random_bases(N)

    pk, _ = bb84(alice_bits, alice_bases, bob_bases)
    truth = [one, one, '-', one, '-', one, one, zero, '-', one]
    assert pk == truth


test_1()

Alice Encoded: [sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |1>, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2]
Bob Measures: [|1>, |1>, |0>, |1>, |0>, |1>, |1>, |0>, |0>, |1>]
pk: [|1>, |1>, '-', |1>, '-', |1>, |1>, |0>, '-', |1>]


In [15]:
def test_2():
    alice_bits = [0, 1, 1,
                  0, 1, 1,
                  1, 0, 1,
                  0, 1, 0]

    alice_bases = ['Z', 'Z', 'X',
                   'Z', 'Z', 'Z',
                   'X', 'Z', 'X',
                   'X', 'X', 'Z']

    bob_bases = ['X', 'Z', 'X',
                 'X', 'Z', 'X',
                 'Z', 'Z', 'X',
                 'X', 'X', 'Z']

    pk, _ = bb84(alice_bits, alice_bases, bob_bases)
    truth = ['-', one, one,
             '-', one, '-',
             '-', zero, one,
             zero, one, zero]
    assert pk == truth


test_2()

Alice Encoded: [|0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, |1>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>]
Bob Measures: [|0>, |1>, |1>, |0>, |1>, |0>, |1>, |0>, |1>, |0>, |1>, |0>]
pk: ['-', |1>, |1>, '-', |1>, '-', '-', |0>, |1>, |0>, |1>, |0>]


In [16]:
def test_3():
    alice_bits = [0, 1, 0, 1, 0, 1, 1, 0, 1]

    alice_bases = ['X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z']

    bob_bases = ['Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z']

    pk, _ = bb84(alice_bits, alice_bases, bob_bases)
    truth = ['-', '-', zero, '-', zero, '-', one, '-', one]
    assert pk == truth


test_3()

Alice Encoded: [sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>]
Bob Measures: [|0>, |0>, |0>, |0>, |0>, |1>, |1>, |0>, |1>]
pk: ['-', '-', |0>, '-', |0>, '-', |1>, '-', |1>]


In [17]:
def test_4():
    alice_bits = [0, 1, 0, 1, 0, 1, 1, 0, 1]

    alice_bases = ['X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z']

    bob_bases = ['Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z']

    pk, _ = bb84(alice_bits, alice_bases, bob_bases)
    truth = ['-', '-', zero, '-', zero, '-', one, '-', one]
    assert pk == truth


test_4()

Alice Encoded: [sqrt(2)*|0>/2 + sqrt(2)*|1>/2, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, |0>, |1>, sqrt(2)*|0>/2 - sqrt(2)*|1>/2, sqrt(2)*|0>/2 + sqrt(2)*|1>/2, |1>]
Bob Measures: [|0>, |0>, |0>, |1>, |0>, |1>, |1>, |1>, |1>]
pk: ['-', '-', |0>, '-', |0>, '-', |1>, '-', |1>]
